# 2. Augment the training dataset and train a new model with a larger dataset

<div class="alert alert-block alert-warning">
<b>Prerequisite:</b> This notebook is part of the <a href="https://catalog.workshops.aws/cv-retail">Computer vision for retail inventory workshop</a>. Please follow the workshop instructions before running this notebook.
</div>

<div class="alert alert-block alert-warning">
<b>Notebook Kernel:</b> Please use the Python 3 (Data Science) kernel to run this notebook.
</div>

<div class="alert alert-block alert-warning">
<b>Execution sequence:</b> Before running this notebook, make sure you have already started <b>1.Train-a-custom-object-detection-model.ipynb</b>. 
</div>

In the previous notebook we trained a custom object detection model using a small dataset of 95 images, out of which, only 66 images were used for training. In this notebook, we will generate **synthetic variations** from these 66 images (called **augmentations**) in order to generate a larger training set. This has the potential to result in an object detection model that generalizes better compared to the previous one, because of the larger variability in the training dataset. 


## Step 0: Dependencies and configuration

Similarly as before, we start by loading useful Python libraries, defining our configuration, and connecting to the AWS SDKs, which will allow us to interface with various AWS services, like Amazon S3 and Amazon SageMaker. 

In [ ]:
# generic packages
import os
import json
import shutil
import imageio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
plt.style.use('seaborn')

# AWS-related packages
import boto3  # AWS Python SDK that allows us to interface with all AWS services
import sagemaker  # SageMaker Python SDK that allows us to easily build train and deploy models

In [ ]:
sagemaker_role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
boto_session = boto3.session.Session()
region = boto_session.region_name


Again, we will be using the **default bucket** of Amazon SageMaker to store our data, model artifacts and model outputs. 

In [ ]:
# variable setup
BUCKET_NAME = sagemaker_session.default_bucket()  # here we will store our data
PREFIX_PROJECT = 'computer-vision-for-retail-workshop'  # main project folder in S3
PREFIX_DATASET = 'dataset-full'  # where our dataset will be located
PREFIX_MODELS = 'models'  # where our trained model weights will be saved
CLASS_NAMES = [  # names of the 10 products that we will be trying to detect
    'flakes',  
    'mm', 
    'coke', 
    'spam', 
    'nutella', 
    'doritos', 
    'ritz', 
    'skittles', 
    'mountaindew', 
    'evian'
]
# print a list of the class names along with their indices
print('Index and class name')
class_indx_names = pd.Series(CLASS_NAMES)
print(class_indx_names,'\n')

MANIFEST_ATTRIBUTE_NAMES = ['source-ref', 'retail-object-labeling']  # attributes to be considered in the manifest files
LOCAL_DATASET_FOLDER = 'dataset'

print('Region:', region)
print('Bucket:', BUCKET_NAME)

# Initialize some empty variables we need to exist:
predictor_std = None
predictor_hpo = None


## Step 1: Explore the image augmentation function

We will be using an image augmentation function `augmentations.py`, located in the `util` folder. The function takes as input an image and generates a set of new images using random transformations, like zooming/unzooming, cropping, sheering, rotation, brightness adjustment, noise, flipping etc. 

Take some time to **explore the impact of each augmentation parameter** in the example below. Here is a quick description for each one. 

| Parameter | Description | Type | Range | Example values and behavior | How to deactivate | 
| --- | --- | --- | --- | --- | --- |
| `how_many` | How many image variations to generate from a single source image | Number | int [0,inf) | e.g. 10 | 0 |
| `random_seed` | A number that controls randomness (for reproducibility) | None or Number| int (-inf, inf) | e.g. 0 | None |
| `range_scale` | Minimum and maximum range for zooming/unzooming on the image | None or Tuple (min,max) | float (0,inf) | <1=zoom in, >1=zoom out, e.g. (0.5,1.5) | None |
| `range_translation` | Minimum and maximum range for offsetting the (x,y) position of the image (in pixels) | None or Tuple (min,max) | int [0,inf) | e.g. (-100, 100) | None |
| `range_rotation` | Minimum and maximum range for rotating image left/right (in degrees) | None or Tuple (min,max) | float [-360,360] | e.g.(-45, 45) | None |
| `range_sheer` | Minimum and maximum range for skewing image left/right (in degrees) | None or Tuple (min,max) | float [-360,360] | e.g.(-45, 45) | None |
| `range_noise` | Minimum and maximum range of noise variance | None or Tuple (min,max) | float [0, inf) | e.g. (0, 0.001) | None |
| `range_brightness` | Minimum and maximum range for brightness gain | None or Tuple (min,max) | float (0, inf) | 1=no change, <1=darken, >1=brighten, e.g. (0.5, 1.5) | None |
| `flip_lr` | Flipping image left-right | None or String | None / 'random' / 'all' | If 'all', all images are doubled (flipped + original). If 'random', images are flipped randomly. | None |
| `flip_ud` | Flipping image up-down| None or String | None / 'random' / 'all' | If 'all', all images are doubled (flipped + original). If 'random', images are flipped randomly. | None |
| `bbox_truncate` | Truncate bounding boxes that may end up outside the augmented image| Boolean | False/True | e.g. True | False |
| `bbox_discard_thr` | Percentage of bounding box surface to be located inside the image, in order not to be discarded | Number | float [0,1] | e.g. 0.85 | N/A |
| `display` | Display augmentations or not in the notebook | Boolean | False/True | Use True only for testing! | False |


You need to come up with a *plausible set of variations* and ranges that makes sense for the current use case. For example, flipping the image upside down is not useful in our case, because it is highly unlikely that we will encounter these retail products upside down (particularly for bottles). Additionally, rotating the image too much, will also result in unrealistic images that cannot be encountered in real life.

In [ ]:
%load_ext autoreload
%autoreload 2

from util.augmentations import augment_affine

filename = 'dataset/dataset-test/IMG_1599.jpg'
ls_bboxes = [{"class_id": 4, "top": 169, "left": 357, "height": 70, "width": 59}, {"class_id": 4, "top": 662, "left": 171, "height": 82, "width": 55}, {"class_id": 8, "top": 85, "left": 256, "height": 153, "width": 60}, {"class_id": 9, "top": 71, "left": 183, "height": 167, "width": 49}, {"class_id": 7, "top": 343, "left": 351, "height": 89, "width": 77}, {"class_id": 5, "top": 372, "left": 244, "height": 65, "width": 113}, {"class_id": 5, "top": 363, "left": 143, "height": 74, "width": 112}, {"class_id": 1, "top": 500, "left": 270, "height": 91, "width": 96}, {"class_id": 1, "top": 497, "left": 173, "height": 96, "width": 99}, {"class_id": 0, "top": 619, "left": 241, "height": 128, "width": 99}, {"class_id": 6, "top": 640, "left": 353, "height": 106, "width": 82}]


# TODO: experiment with different augmentation parameters and test their impact. 

image_augm = augment_affine(
    image_filename=filename,        # the image that will be used as source to generate augmentations
    bboxes =ls_bboxes,              # a list of bounding boxes in the source image
    how_many=10,                    # how many image variations to generate from the source image
    random_seed=0,                  # controls randomness for reproducibility
    range_scale=(0.75, 1.5),        # (multiplier) minimum and maximum range for zooming/unzooming on the image
    range_translation=(-50, 50),    # (in pixels) minimum and maximum range for offsetting the position of the image 
    range_rotation=(-5, 5),         # (in degrees) minimum and maximum range for rotating image left/right.
    range_sheer=(-5, 5),            # (in degrees) minimum and maximum range for skewing image left/right.
    range_noise=(0, 0.001),         # (variance) minimum and maximum range for noise variance
    range_brightness=(0.8, 1.5),    # (multiplier) minimum and maximum range for brightness gain
    flip_lr='random',               # If None, no left-right flipping is applied. If 'all', all images are flipped. If 'random', images are flipped randomly
    flip_ud=None,                   # same as flip_lr, but for up-down.
    bbox_truncate = True,           # truncate bboxes that may end up outside the augmented image.
    bbox_discard_thr = 0.85,        # percentage of bounding box surface to be located inside the image, in order not to be discarded. 
    display=True                    # display augmentations (set as False if you are generating lots of images!!!)
    )


## Step 2: Augment the training dataset

Based on your experimentation, select the final range of values that you would like your augmentations to have. These parameters will be applied when generating the augmented images from the original ones. Also, be mindful of the `AUGM_PER_IMAGE` parameter, because it can increase the dataset a lot (acts as a multiplier in the total number of images) and thus, result in longer training times. 

Here are some suggestions, but feel free to use your own! 

- `AUGM_PER_IMAGE = 5`
- `RANDOM_SEED = 0`
- `RANGE_SCALE = (0.75, 1.5)`
- `RANGE_TRANSLATION = (-50, 50)`
- `RANGE_ROTATION = (-5, 5)`
- `RANGE_SHEER = (-5, 5)`
- `RANGE_NOISE = (0, 0.001)`
- `RANGE_BRIGHTNESS = (0.8, 1.5)`
- `FLIP_LR = 'random'` 
- `FLIP_UD = None`

Replace the `?` with your own values. After adding your chosen values, continue to run the rest of the notebook. Optionally, you can also choose to use "Run Selected Cell and All Below". 

In [ ]:
# TODO: select the range of augmentations that you want to apply

AUGM_PER_IMAGE = ?            # how many augmentations to generate from each image (increases the whole dataset size by this factor)
RANDOM_SEED = ?               # control randomness for reproducibility
RANGE_SCALE = (?, ?)          # (multiplier) minimum and maximum range for zooming/unzooming on the image
RANGE_TRANSLATION = (?, ?)    # (in pixels) minimum and maximum range for offsetting the position of the image 
RANGE_ROTATION = (?, ?)       # (in degrees) minimum and maximum range for rotating image left/right.
RANGE_SHEER = (?, ?)          # (in degrees) minimum and maximum range for skewing image left/right.
RANGE_NOISE = (?, ?)          # (variance) minimum and maximum range for noise variance
RANGE_BRIGHTNESS = (?, ?)     # (multiplier) minimum and maximum range for brightness gain
FLIP_LR = ?                   # mirror image left right
FLIP_UD = ?                   # mirror image up down


In [ ]:
# initialize a local folder to store the augmented images

LOCAL_AUGMENTED_TRAINSET_FOLDER = f'{LOCAL_DATASET_FOLDER}/dataset-augmented'  
if os.path.exists(LOCAL_AUGMENTED_TRAINSET_FOLDER) is False:
    os.makedirs(LOCAL_AUGMENTED_TRAINSET_FOLDER)
    

We will now generate the augmented images. First we will load the updated training manifest file (created in the 1st notebook) and use it as a guide. From each JSON line (which essentially is one training image), we will generate multiple augmented images, along with their new updated bounding boxes. At the same time, we will start populating a new training manifest file, which will include *both* the original training images and the augmented ones. In the meantime, we will also count the number of training examples generated per class. This can take 2-3 minutes to complete.

In [ ]:
# Augment the whole training dataset

from pathlib import Path

new_manifest = []
n_samples_training_augmented = 0
n_samples_training_original = 0
class_histogram_train_original = np.zeros(len(CLASS_NAMES), dtype=int)
class_histogram_train_augmented = np.zeros(len(CLASS_NAMES), dtype=int)

with open(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/train-updated.manifest') as f:  # open the manifest file
    lines = f.readlines()
    
for line in lines:
    line_dict = json.loads(line)  # load one json line (corresponding to one image)
    filename_object = Path(line_dict['source-ref'])
    filename = str(filename_object.name)  # filename without the path
    
    # add json line of the original image and count the examples inside it
    new_manifest.append(json.dumps(line_dict))
    n_samples_training_original += 1
    for j,annotation in enumerate(line_dict['retail-object-labeling']['annotations']):
        class_histogram_train_original[int(line_dict['retail-object-labeling']['annotations'][j]['class_id'])] += 1  # counting annotations
    
    # generate augmented images
    print('Augmenting image:',filename)
    image_augm = augment_affine(
        image_filename=f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/{filename}',
        bboxes =line_dict['retail-object-labeling']['annotations'],
        how_many=AUGM_PER_IMAGE,
        random_seed=RANDOM_SEED,
        range_scale=RANGE_SCALE,
        range_translation=RANGE_TRANSLATION,
        range_rotation=RANGE_ROTATION,
        range_sheer=RANGE_SHEER,
        range_noise=RANGE_NOISE,     
        range_brightness=RANGE_BRIGHTNESS,   
        flip_lr=FLIP_LR,
        flip_ud=FLIP_UD,
        bbox_truncate = True,
        bbox_discard_thr = 0.85,
        display=False  # otherwise the notebook will be flooded with images!
    )
    
    # save augmented images locally
    for i,image in enumerate(image_augm['Images']):
        
        # new image size of augmented image
        image_height = image.shape[0]
        image_width = image.shape[1]
        if len(image.shape) == 3:
            image_depth = image.shape[2]
        else:
            image_depth = 1
        line_dict['retail-object-labeling']['image_size'] = [{"width": image_width, "height": image_height, "depth": image_depth}]
        
        # augmented image filename
        filename_no_extension = str(filename_object.stem)  # filename without extension 
        filename_augmented = f'{filename_no_extension}_augm_{str(i+1)}.jpg'
        image_augm_filename = f'{LOCAL_AUGMENTED_TRAINSET_FOLDER}/{filename_augmented}'
        imageio.imsave(image_augm_filename, image, quality=95)  # save locally
        new_filename_s3 = f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/{filename_augmented}'
        line_dict['source-ref'] = new_filename_s3  # add new filename to the manifest file
        
        # new image bounding boxes
        line_dict['retail-object-labeling']['annotations'] = image_augm['bboxes'][i]
        
        # add a new json line for this augmentation image
        new_manifest.append(json.dumps(line_dict))
        
        n_samples_training_augmented += 1  # count training images
        for j,annotation in enumerate(line_dict['retail-object-labeling']['annotations']):
            class_histogram_train_augmented[int(line_dict['retail-object-labeling']['annotations'][j]['class_id'])] += 1  # count annotations
    
    
# save the updated training manifest file locally
with open(f"{LOCAL_AUGMENTED_TRAINSET_FOLDER}/train-augmented.manifest", "w") as f:
    for line in new_manifest:
        f.write(f"{line}\n")  

Let's compare now the size of the augmented dataset to the original one. Remember that the augmented training dataset includes both the original training images plus the generated ones. 

In [ ]:
# depict statistics about the existing dataset splits

print('Oroginal training images:', n_samples_training_original)
print('Augmented training images:', n_samples_training_original + n_samples_training_augmented)

df_dataset_stats = pd.DataFrame(
    {
        'train_augmented': class_histogram_train_augmented + class_histogram_train_original, 
        'train_original': class_histogram_train_original, 
        'class': CLASS_NAMES
    }
) 

df_dataset_stats.plot.bar(x='class')
plt.title('Number of examples per dataset')
plt.show()


We see that now we have many more training examples per class, as well as more overall training images. We need now to copy these images and the new manifest file to Amazon S3, in order to use them for training. 

In [ ]:
# copy augmented manifest file to S3
sagemaker_session.upload_data(
    path=f'{LOCAL_AUGMENTED_TRAINSET_FOLDER}/train-augmented.manifest',
    bucket=BUCKET_NAME,
    key_prefix=f'{PREFIX_PROJECT}/{PREFIX_DATASET}'
)

# copy all augmented images to the rest of the dataset in S3
import glob
ls_augmented_images = glob.glob(f'{LOCAL_AUGMENTED_TRAINSET_FOLDER}/*.jpg')

for i,filename in enumerate(ls_augmented_images):
    print('Copying augmented image', i+1, 'out of', len(ls_augmented_images), '...\r', end='')
    sagemaker_session.upload_data(
        path=filename,
        bucket=BUCKET_NAME,
        key_prefix=f'{PREFIX_PROJECT}/{PREFIX_DATASET}'
    )
    

## Step 3: Set up input data channels

In order for SageMaker to train a machine learning model, it needs to know where the training and validation datasets are located. In SageMaker language, this is called **input channels**. Input channels are **objects** containing the location of the datasets in S3, along with the type of data and annotations. 

In our case we have in S3, both for the training and validation datasets:

* A **JSONLines manifest file** listing what images are in the dataset (by their S3 URI) and what annotations have been collected for those images (bounding boxes + class names)
* The image files themselves

We want SageMaker to provide the algorithm with a **stream of image records** comprising both the image data and their annotations. This will be faster compared to downloading the full dataset to the training container. The [algorithm docs](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html#object-detection-inputoutput) give guidance on how to set this up: SageMaker already provides functionality to create RecordIO files from manifest files. 

In [ ]:
train_channel = sagemaker.inputs.TrainingInput(
    f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/train-augmented.manifest',  # we are using the new AUGMENTED training manifest file
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    s3_data_type="AugmentedManifestFile",
    record_wrapping="RecordIO",
    attribute_names=MANIFEST_ATTRIBUTE_NAMES,
    shuffle_config=sagemaker.inputs.ShuffleConfig(seed=1)
)
                                        
validation_channel = sagemaker.inputs.TrainingInput(
    f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/validation-updated.manifest',  # we are using the same validation manifest file
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    record_wrapping="RecordIO",
    s3_data_type="AugmentedManifestFile",
    attribute_names=MANIFEST_ATTRIBUTE_NAMES,
)

## Step 4: Configure the algorithm

The first step in deciding to use a SageMaker built-in algorithm is to review its [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html) and [hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection-api-config.html). In particular we'll need the **URL for the Docker image** in order to use a built-in algorithm. While this is listed [in the docs](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html), it's also nice and easy to fetch programmatically.


In [ ]:
training_image = sagemaker.image_uris.retrieve(
    region=region, 
    framework="object-detection", 
    version="1"  # or use "latest"
)
print('Container image:', training_image)


The remainder of configuration includes:

* Setting where to store final model artifacts and intermediate checkpoints
* Specifying which compute resource to use
* Selecting the algorithm's hyperparameters

We do this through the [SageMaker SDK's Estimator API](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html), similarly to estimators in other common frameworks. Some things to keep in mind:

* [Pipe Mode](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html#cdf-pipe-mode) streams input data to the algorithm rather than (the default) downloading the whole dataset up-front. This can accelerate training start-up for algorithms that support it.
* The Object Detection built-in algorithm supports GPU-accelerated and distributed training. Here we use a GPU-accelerated `ml.p3.2xlarge` instance. There is no need for distributed training (more than one instance),  due to the small dataset size.

In [ ]:
estimator = sagemaker.estimator.Estimator(
    training_image,  # URL to container image implementing the algorithm 
    sagemaker_role,  # IAM access to perform the API actions
    input_mode="Pipe",  # or "File" mode
    instance_count=1,  # if more than 1, then we have distributed training (not needed here!)
    instance_type="ml.p3.2xlarge",  # type of instance to be used for training
    volume_size=50,  # (in GB) storage volume to use for storing input and output data during training 
    max_run=10*60*60,  # (in sec) maximum time of the training job
    output_path=f"s3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_MODELS}"  # where to store the model weights
)


Next we will select the [algorithm's hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection-api-config.html). Here we selected a promising set of hyperparameters, after performing some initial testing. 

Since the size of the training set is now larger (depending on the number of augmentations you introduced), you may be able to train the model from scratch (using random initial weights), without starting from pretrained ones. This can be done by setting `use_pretrained_model=0` in the hyperparameters. In this case however, training will take much longer; the optimizer will need more epochs (~50-100) to reach a plateau, and therefore, more waiting time. 

For this reason, and in the interest of time, we suggest to still use pretrained weights (by setting `use_pretrained_model=1`). This will allow the optimizer to converge faster. In fact, you will later observe that the optimizer reaches a plateau in fewer than 10 epochs, whereas before, without augmentations, it needed at least 20 epochs.

In [ ]:
estimator.set_hyperparameters(
    base_network="resnet-50",  # or 'vgg-16' architecture to be used
    use_pretrained_model=1,  # 0/1 whether to use pretrained or random weights (0-> train from scratch) 
    early_stopping = True,
    early_stopping_min_epochs = 5,
    early_stopping_patience = 3,
    early_stopping_tolerance = 0.00,
    num_classes=len(CLASS_NAMES),
    mini_batch_size=8,  # depends on the GPU memory and the image sizes
    epochs=20,
    learning_rate=0.00005,  # very small learning rate to avoid catastrophic forgeting
    lr_scheduler_step="20,40,60,80",
    lr_scheduler_factor=0.5,
    optimizer="adam",
    momentum=0.2834,
    weight_decay=0.94,
    overlap_threshold=0.5,
    nms_threshold=0.45,
    image_shape=832,
    label_width=350,
    num_training_samples=n_samples_training_original + n_samples_training_augmented,
)


## Step 5: Train the model

The hyperparameters above represent our best up-front guess; and it's easy enough to call `estimator.fit()` to train a model as shown below.

One way to improve model performance and reduce some of the guesswork, is to let SageMaker `HyperParameterTuner` optimize them. SageMaker Hyper-Parameter Optiomization (HPO) supports [Random, Grid, Bayesian and Hyperband strategies](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html)). In this example we prefer the [Hyperband approach, which usually exhibits a more competitive performance compared to ther rest](https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-automatic-model-tuning-now-provides-up-to-three-times-faster-hyperparameter-tuning-with-hyperband/) and faster convergance. Because HPO typically takes much longer than standard model fitting, `tuner.fit()` is an **asynchronous** method by default whereas `estimator.fit()` is **synchronous** (blocking).

<div class="alert alert-block alert-warning">
<b>Notice:</b> The following training code will take <b>approximately 12-15 minutes</b> to execute (without HPO). If you opt to activate HPO, it will take 2-4 hours (depending on how much data you added through augmentation)! Therefore, we do not recommend running HPO during the workshop, but you are welcome to try it in your own time. 
</div>

While you wait for the training to finish, why not read up on SageMaker using any of the links in this notebook? Or, if you don't like reading, watch this cool under-the-hood video about [Amazon Go](https://www.youtube.com/watch?v=Lu4szyPjIGY).

In [ ]:
WITH_HPO = False  # change to True if you want to find better parameters (attention! it takes long time!)

In [ ]:
%%time

if WITH_HPO is False:
    estimator.fit(
        { "train": train_channel, "validation": validation_channel }, 
        logs=True
    )
    
else:
    hyperparameter_ranges = {
        "learning_rate": sagemaker.tuner.ContinuousParameter(0.00001, 0.01),
        "momentum": sagemaker.tuner.ContinuousParameter(0.0, 0.99),
        "weight_decay": sagemaker.tuner.ContinuousParameter(0.0, 0.99),
        "optimizer": sagemaker.tuner.CategoricalParameter(["sgd", "adam", "rmsprop", "adadelta"])
    }

    tuner = sagemaker.tuner.HyperparameterTuner(
        estimator,
        "validation:mAP",  # Name of the objective metric to optimize. "Mean Average Precision" high = good
        objective_type="Maximize",  # or Minimize
        strategy="Hyperband", # or Baysian or Random. 
        hyperparameter_ranges=hyperparameter_ranges,
        base_tuning_job_name="object-detection-SSD-HPO",
        max_jobs=50,  # how many searches (training jobs) we will have in the parameter space
        max_parallel_jobs=1  # how many searches (training jobs) will happen in parallel
    )
    
    tuner.fit(
        { "train": train_channel, "validation": validation_channel },
        include_cls_metadata=False
    )

Once training finishes, we can explore the training logs and see how performance on the validation set changed over time through the training epochs.


In [ ]:
import boto3
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline
plt.style.use('seaborn')

client = boto3.client("logs")
BASE_LOG_NAME = "/aws/sagemaker/TrainingJobs"

def plot_object_detection_log(model, title):
    # retrieve from the training job logs the mAP across epochs and plot it
    logs = client.describe_log_streams(
        logGroupName=BASE_LOG_NAME, logStreamNamePrefix=model._current_job_name
    )
    cw_log = client.get_log_events(
        logGroupName=BASE_LOG_NAME, logStreamName=logs["logStreams"][0]["logStreamName"]
    )

    mAP_accs = []
    for e in cw_log["events"]:
        msg = e["message"]
        if "validation mAP <score>=" in msg:
            num_start = msg.find("(")
            num_end = msg.find(")")
            mAP = msg[num_start + 1 : num_end]
            mAP_accs.append(float(mAP))

    print(title)
    print("Maximum mAP: %f " % max(mAP_accs))

    fig, ax = plt.subplots()
    plt.xlabel("Epochs")
    plt.ylabel("Mean Avg Precision (mAP)")
    plt.title("Validation performance per training epoch")
    (val_plot,) = ax.plot(range(len(mAP_accs)), mAP_accs, label="mAP")
    plt.legend(handles=[val_plot])
    ax.yaxis.set_ticks(np.arange(0.0, 1.05, 0.1))
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%0.2f"))
    plt.show()
 
    
if WITH_HPO is True: 
    estimator = tuner.best_estimator()
    
plot_object_detection_log(estimator, "mAP tracking for job: " + estimator._current_job_name)

## Step 6: Deploy the model

Once the model is trained, SageMaker supports many different [deployment options](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html). In this example we'll deploy our trained model to a **real-time endpoint**, in order to have the lowest possible latency in our predictions. You can think an endpoint as a dedicated web-server, making accessible our trained model's predictions through a REST API. Since our endpoints won't be handling any significant traffic volumes, we provision a single non-accelerated instance.

<div class="alert alert-block alert-warning">
<b>Notice:</b> The following deployment of the model to a realtime endpoint will take approximately <b>5-7 minutes</b>.
</div>

<div class="alert alert-block alert-warning">
<b>Attention:</b> Please copy the <b>name of the deployed endpoint</b> (in the output of the following cell). You will need to paste it in the application. This will allow the application to route camera images to your endpoint for real-time inference.
</div>

In [ ]:
%%time
if WITH_HPO is True:
    if (predictor_hpo):  # in case HPO was selected
        try:
            predictor_hpo.delete_endpoint()
            print("Deleted previous HPO endpoint")
        except:
            print("Couldn't delete previous HPO endpoint")
    print("Deploying HPO model...")
    predictor_hpo = tuner.deploy(
        initial_instance_count=1,
        instance_type="ml.c5.large",
        # wait=False,
    )
else:
    if (predictor_std):  # in case simple training was selected
        try:
            predictor_std.delete_endpoint()
            print("Deleted previous non-HPO endpoint")
        except:
            print("Couldn't delete previous non-HPO endpoint")
    print("Deploying standard (non-HPO) model...")
    predictor_std = estimator.deploy(
        initial_instance_count=1,  # number of instances for the endpoint
        instance_type="ml.c5.large",  # type of instance to be used for the endpoint
        # wait=False,
    )

predictor = predictor_hpo if WITH_HPO else predictor_std
print('Name of the deployed endpoint:',predictor.endpoint_name)

<div class="alert alert-block alert-warning">
<b>Attention:</b> Please copy the <b>name of the deployed endpoint</b> (in the output of the previous cell). You will need to paste it in the application. This will allow the application to route camera images to your endpoint for real-time inference.
</div>

## Step 7: Run inference on test images

Now that we have one object detection model deployed, we can send some test images and see how it performs!


#### Test on individual images

The `predict()` function used here, provided in the `util` folder, includes code that allows us to send image files to our deployed endpoint and receive back the detection results. 

The built-in Object Detection algorithm doesn't estimate an optimal confidence threshold for us. Instead, it returns **all the detections**, irrespective of their confidence score. The `predict()` function includes the confidence threshold parameter `thresh`. Any detection with confidence score higher than the `thresh` parameter, will be visualized. The function also returns a Pandas DataFrame with all the detected bounding boxes for the given detection threshold. 

For visualization, the `predict()` function calls the `visualize_detection()` function, which is uses Matplotlib to plot the provided detection boxes over the image. Test different filenames from the test folder, as well as different `thresh` values.

In [ ]:
%load_ext autoreload
%autoreload 2

from util.util import predict

sagemaker_runtime = boto3.client(service_name="runtime.sagemaker")
endpoint_name = predictor.endpoint_name
print('Using endpoint:',endpoint_name)

df_results = predict(
    # change this to other test images to see more results
    filename=f'{LOCAL_DATASET_FOLDER}/dataset-test/IMG_1599.jpg',  
    runtime=sagemaker_runtime,
    class_names = CLASS_NAMES,
    endpoint_name=endpoint_name, 
    thresh=0.2, 
    visualize=True
)

df_results


#### Compare multiple detection thresholds

You can even generate side by side comparison of the results of multiple detection thresholds, by providing a **list of thresholds**, instead of a single number. In the following example, we depict the results for 3 different detection threshold values `[0.3, 0.6, 0.9]`.

In [ ]:
%load_ext autoreload
%autoreload 2

from util.util import predict

sagemaker_runtime = boto3.client(service_name="runtime.sagemaker")
endpoint_name = predictor.endpoint_name
print('Using endpoint:',endpoint_name)

df_results = predict(
    # change this to other test images to see more results
    filename=f'{LOCAL_DATASET_FOLDER}/dataset-test/IMG_1599.jpg',  
    runtime=sagemaker_runtime,
    class_names = CLASS_NAMES,
    endpoint_name=endpoint_name, 
    thresh=[0.3,0.6,0.9],  # a list of thresholds to be visualized
    visualize=True
)


One thing that is evident, is that with the augmented training dataset, the model makes more confident decisions compared to when it was trained with the original smaller dataset. 

#### Test on the whole test set

We can now call the `evaluate_testset()` helper function, which will test the deployed model against the **whole test set** and generate a performance report, for each class. The helper function relies on two important parameters. **Intersection Over Union (IOU)** and the model's confidence threshold. IOU is a metric that describes the degree of overlap between two bounding boxes, and it is needed in order to know when the output of the model coincides with the ground truth bounding box. Check this page for a more [detailed description on IOU](https://en.wikipedia.org/wiki/Jaccard_index).

In [ ]:
%load_ext autoreload
%autoreload 2

from util.util import evaluate_testset

df_class_performance = evaluate_testset(
    runtime=sagemaker_runtime, 
    endpoint_name=endpoint_name,
    class_names = CLASS_NAMES,
    testset_folder = f'{LOCAL_DATASET_FOLDER}/dataset-test', 
    test_manifest_file = f'{LOCAL_DATASET_FOLDER}/dataset-test/test-updated.manifest', 
    thr_iou = 0.5,  # Intersection Over Union threshold
    thr_conf = 0.2  # Confidence Threshold
)

df_class_performance


Here is a quick explanation for the performance statistics. Check this page for a more [detailed description](https://en.wikipedia.org/wiki/Precision_and_recall).

| Abbreviation | Full name | Explanation | Range | Intuition |
| --- | --- | --- | --- | --- | 
| `TP` | True Positives | How many times the model has correctly detected an actual object | Depends on test set | Higher is better |
| `FP` | False Positives | How many times the model has mistakenly detected something that was not an actual object | Depends on test set  | Lower is better |
| `FN` | False Negatives | How many times the model did not detect (missed) an actual object | Depends on test set | Lower is better |
| `TN` | True Negatives | (**Does not apply in object detection!**) How many times the model correctly did not detect something that was not an actual object | N/A | N/A |
| `PR` | Precision | What percentage of all the model's detections were indeed actual objects. | [0,1] | Higher is better |
| `RE` | Recall | What percentage of all the actual available objects were detected by the model | [0,1] | Higher is better |
| `F1` | F1 score | A balanced combination (harmonic mean) of Precision and Recall.  | [0,1] | Higher is better |

In [ ]:
# plot a graph of the results 

plt.figure()
df_class_performance.plot.bar(x='CLASS')
plt.title('Model performance on the Testset')
plt.show()


We see that performance is better compared to when the model was trained with the original smaller training dataset (Notebook 1). 

## Step 8: Clean up

Although training instances are ephemeral, the resources we allocated for real-time endpoints need to be cleaned up to avoid ongoing charges. The code below will delete the *most recently deployed* endpoint for the HPO and non-HPO configurations, but note that if you deployed either more than once, you might end up with extra endpoints. To be safe, it's best to still check through the SageMaker console for any left-over resources when cleaning up.

In [ ]:
# uncomment the following code in order to delete the created resources

# if (predictor_hpo):
#     print("Deleting HPO-optimized predictor endpoint")
#     predictor_hpo.delete_endpoint()
# if (predictor_std):
#     print("Deleting standard (non-HPO) predictor endpoint")
#     predictor_std.delete_endpoint()

## Review

In this notebook we augmented the initial small training dataset, by generating sythetic transformations from each training image (augmentations). This resulted in a larger training dataset, that covered more variations in the position, size and orientation of the objects. We saw that the resulting model gives more confident decisions and achieved a better performance in our hold out testing dataset. 
